# Account Routes


In [ ]:
# | default_exp routes.account

In [ ]:
# | exporti
from typing import Union
from enum import Enum
import httpx
import asyncio
import datetime as dt

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de

import domolibrary.classes.DomoAccount_Config as dmac

In [ ]:
# |hide

from nbdev import show_doc

# Account Route Error Types


# Retrieval Routes


In [ ]:
# | export
@gd.route_function
async def get_accounts(
    auth: dmda.DomoAuth,
    debug_api: bool = False,
    debug_num_stacks_to_drop=1,
    parent_class: str = None,
    session: httpx.AsyncClient = None,
) -> rgd.ResponseGetData:
    """retrieve a list of all the accounts the user has read access to.  Note users with "Manage all accounts" will retrieve all account objects"""

    url = f"https://{auth.domo_instance}.domo.com/api/data/v1/accounts"

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="GET",
        debug_api=debug_api,
        num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
        session=session,
    )
    return res

#### sample implementation of get_accounts


In [ ]:
show_doc(get_accounts)

---

[source](https://github.com/jaewilson07/domo_library/blob/main/domolibrary/routes/account.py#L26){target="_blank" style="float:right; font-size:smaller"}

### get_accounts

>      get_accounts (auth:domolibrary.client.DomoAuth.DomoAuth,
>                    debug_api:bool=False, debug_num_stacks_to_drop=1,
>                    parent_class:str=None, session:httpx.AsyncClient=None)

retrieve a list of all the accounts the user has read access to.  Note users with "Manage all accounts" will retrieve all account objects

In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)


res = await get_accounts(auth=token_auth)
pd.DataFrame(res.response)

,id,userId,name,displayName,type,valid,dataProviderType,credentialsType,createdAt,createdBy,modifiedAt,modifiedBy,configurations,accountTemplateAuthorizationId,accountId,accountTemplateId
0,1,1893952720,DataSet Copy Account,dsa - northshore,data,True,dataset-copy,fields,1589100087000,1893952720,1698071224000,1893952720,{},None,1,None
1,5,1893952720,Domo Governance Datasets Third Party Account,test_rename,data,True,domo-governance-d14c2fef-49a8-4898-8ddd-f64998...,fields,1616777681000,1893952720,1699633936000,1893952720,{},None,5,None
2,27,1893952720,DataSet Copy Account,DataSet Copy Account,data,True,domo-csv,fields,1619083568000,1893952720,1619083568000,1893952720,{},None,27,None
3,45,1893952720,onyxReporting@gmail.com,test-goolesheets,data,True,google-spreadsheets,oauth,1664924354000,1893952720,1682350828000,1893952720,{},None,45,None
4,70,1893952720,jaemyong.wilson@sony.com,jaemyong.wilson@sony.com,data,True,google-spreadsheets,oauth,1682511651000,1893952720,1682511651000,1893952720,{},None,70,None
5,71,1893952720,domo_creds,domolibrary test account - updated 2023-11-10,data,True,abstract-credential-store,fields,1684447092000,1893952720,1699637388000,1893952720,{},None,71,None


In [ ]:
# | export
class GetAccount_NoMatch(de.DomoError):
    def __init__(
        self,
        domo_instance,
        account_id=None,
        status=None,
        function_name=None,
        parent_class=None,
    ):
        message = f"account_id {account_id} not found"

        super().__init__(
            message=message,
            status=status,
            function_name=function_name,
            parent_class=parent_class,
            domo_instance=domo_instance,
        )


class GetAccount_NoConfigRetrieved(de.DomoError):
    def __init__(
        self,
        account_id,
        domo_instance,
        status=None,
        function_name=None,
        parent_class=None,
    ):
        message = f"account_id {account_id} did not return a config.  update `DomoAccount_Config` if it uses OAuth, otherwise this is probably an error"

        super().__init__(
            message=message,
            status=status,
            function_name=function_name,
            parent_class=parent_class,
            domo_instance=domo_instance,
        )


class DeleteAccount_Error(de.DomoError):
    def __init__(
        self,
        entity_id,
        domo_instance,
        status,
        message,
        function_name=None,
        parent_class=None,
    ):
        super().__init__(
            entity_id=entity_id,
            domo_instance=domo_instance,
            status=status,
            message=message,
            function_name=function_name,
            parent_class=parent_class,
        )

In [ ]:
# | export
@gd.route_function
async def get_account_from_id(
    auth: dmda.DomoAuth,
    account_id: int,
    debug_api: bool = False,
    debug_num_stacks_to_drop: int = 1,
    parent_class: str = None,
    session: httpx.AsyncClient = None,
) -> rgd.ResponseGetData:
    """retrieves metadata about an account"""

    url = f"https://{auth.domo_instance}.domo.com/api/data/v1/accounts/{account_id}?unmask=true"

    if debug_api:
        print(url)

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="GET",
        debug_api=debug_api,
        session=session,
        timeout=20,  # occasionally this API has a long response time
        num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=None,
    )

    if not res.is_success and (
        res.response == "Forbidden" or res.response == "Not Found"
    ):
        raise GetAccount_NoMatch(
            account_id=account_id,
            domo_instance=auth.domo_instance,
            status=res.status,
            parent_class=parent_class,
            function_name=res.traceback_details.function_name,
        )

    return res

#### sample implementation of get_account_from_id


In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

res = await get_account_from_id(auth=token_auth, account_id=45)
pd.DataFrame([res.response])

,id,userId,name,displayName,type,valid,dataProviderType,credentialsType,createdAt,createdBy,modifiedAt,modifiedBy,configurations,accountTemplateAuthorizationId,accountId,accountTemplateId
0,45,1893952720,onyxReporting@gmail.com,test-goolesheets,data,True,google-spreadsheets,oauth,1664924354000,1893952720,1682350828000,1893952720,{},None,45,None


# Account Config


In [ ]:
# | export


@gd.route_function
async def get_account_config(
    auth: dmda.DomoAuth,
    account_id: int,
    return_raw: bool = False,
    debug_api: bool = False,
    debug_num_stacks_to_drop: int = 1,
    parent_class: str = None,
    session: Union[httpx.AsyncClient, httpx.AsyncClient, None] = None,
) -> rgd.ResponseGetData:
    res = await get_account_from_id(
        auth=auth,
        account_id=account_id,
        debug_api=debug_api,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
        session=session,
    )

    data_provider_type = res.response.get("dataProviderType")
    url = f"https://{auth.domo_instance}.domo.com/api/data/v1/providers/{data_provider_type}/account/{account_id}?unmask=true"

    if debug_api:
        print(url)

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="GET",
        debug_api=debug_api,
        session=session,
        parent_class=parent_class,
        num_stacks_to_drop=debug_num_stacks_to_drop,
    )

    if return_raw:
        return res

    if not res.is_success:
        raise GetAccount_NoMatch(
            account_id=account_id,
            domo_instance=auth.domo_instance,
            status=res.status,
            parent_class=parent_class,
            function_name=res.traceback_details.function_name,
        )

    res.response.update(
        {
            "_search_metadata": {
                "account_id": account_id,
                "data_provider_type": data_provider_type,
            }
        }
    )

    return res

#### sample implementation of get_account_config

example with uses OAuth


In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

res = await get_account_config(
    auth=token_auth,
    account_id=45,
    debug_api=False,
    return_raw=False,
)

res.response

{'_search_metadata': {'account_id': 45,
  'data_provider_type': 'google-spreadsheets'}}

standard config response


In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

await get_account_config(
    auth=token_auth,
    account_id=45,
    debug_api=False,
    return_raw=False,
)

ResponseGetData(status=200, response={'_search_metadata': {'account_id': 45, 'data_provider_type': 'google-spreadsheets'}}, is_success=True, parent_class=None)

In [ ]:
# | export


@gd.route_function
async def get_user_access(
    auth: dmda.DomoAuth,
    account_id: int,
    return_raw: bool = False,
    debug_api: bool = False,
    debug_num_stacks_to_drop: int = 1,
    parent_class: str = None,
    session: Union[httpx.AsyncClient, httpx.AsyncClient, None] = None,
) -> rgd.ResponseGetData:
    res = await get_account_from_id(
        auth=auth,
        account_id=account_id,
        debug_api=debug_api,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
        session=session,
    )

    data_provider_type = res.response.get("dataProviderType")
    url = f"https://{auth.domo_instance}.domo.com/api/data/v1/providers/{data_provider_type}/account/{account_id}?unmask=true"

    if debug_api:
        print(url)

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="GET",
        debug_api=debug_api,
        session=session,
        parent_class=parent_class,
        num_stacks_to_drop=debug_num_stacks_to_drop,
    )

    if return_raw:
        return res

    if not res.is_success:
        raise GetAccount_NoMatch(
            account_id=account_id,
            domo_instance=auth.domo_instance,
            status=res.status,
            parent_class=parent_class,
            function_name=res.traceback_details.function_name,
        )

    res.response.update(
        {
            "_search_metadata": {
                "account_id": account_id,
                "data_provider_type": data_provider_type,
            }
        }
    )

    return res

# CRUD Routes


In [ ]:
# | export


class UpdateAccount_Error(de.DomoError):
    def __init__(
        self,
        status,
        response,
        account_id,
        domo_instance,
        info=None,
        function_name: str = None,
        parent_class: str = None,
    ):
        message = f"unable to update account {account_id} - {response} { (' - ' + info) or ''}"

        super().__init__(
            status=status,
            message=message,
            domo_instance=domo_instance,
            parent_class=parent_class,
            function_name=function_name,
        )


@gd.route_function
async def update_account_config(
    auth: dmda.DomoAuth,
    account_id: int,
    config_body: dict,
    debug_api: bool = False,
    debug_num_stacks_to_drop: int = 1,
    parent_class: str = None,
    session: httpx.AsyncClient = None,
) -> rgd.ResponseGetData:
    # get the data_provider_type, which is necessare for updating the config setting
    res = await get_account_from_id(
        auth=auth,
        account_id=account_id,
        debug_api=debug_api,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
        session=session,
    )
    data_provider_type = res.response.get("dataProviderType")
    url = f"https://{auth.domo_instance}.domo.com/api/data/v1/providers/{data_provider_type}/account/{account_id}"

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="PUT",
        body=config_body,
        debug_api=debug_api,
        num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
        session=session,
    )

    if res.status == 400 and res.response == "Bad Request":
        raise UpdateAccount_Error(
            status=res.status,
            response=res.response,
            account_id=account_id,
            info="updating config | use debug_api to check the URL - ",
            domo_instance=auth.domo_instance,
        )

    if res.status != 200:
        raise UpdateAccount_Error(
            status=res.status,
            response=res.response,
            account_id=account_id,
            info="updating account config",
            domo_instance=auth.domo_instance,
        )

    return res

#### sample implementation of update_account


In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

try:
    res = await update_account_config(
        auth=token_auth,
        account_id=71,
        config_body={"credentials": "abc123"},
        debug_api=False,
    )
    print(res)

except Exception as e:
    print(e)

ResponseGetData(status=200, response={'id': 71, 'userId': 1893952720, 'name': 'domo_creds', 'displayName': 'domolibrary test account - updated 2023-11-10', 'type': 'data', 'valid': True, 'dataProviderType': 'abstract-credential-store', 'credentialsType': 'fields', 'createdAt': 1684447092000, 'createdBy': 1893952720, 'modifiedAt': 1699637388000, 'modifiedBy': 1893952720, 'configurations': {}, 'accountTemplateAuthorizationId': None, 'accountId': 71, 'accountTemplateId': None}, is_success=True, parent_class=None)


In [ ]:
# | export


@gd.route_function
async def update_account_name(
    auth: dmda.DomoAuth,
    account_id: int,
    account_name: str,
    debug_api: bool = False,
    debug_num_stacks_to_drop: int = 1,
    parent_class: str = None,
    session: httpx.AsyncClient = None,
) -> rgd.ResponseGetData:
    url = (
        f"https://{auth.domo_instance}.domo.com/api/data/v1/accounts/{account_id}/name"
    )

    if debug_api:
        print(url)

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="PUT",
        body=account_name,
        content_type="text/plain",
        debug_api=debug_api,
        parent_class=parent_class,
        num_stacks_to_drop=debug_num_stacks_to_drop,
        session=session,
    )

    if res.status != 200:
        raise UpdateAccount_Error(
            status=res.status,
            response=res.response,
            account_id=account_id,
            function_name=res.traceback_details.function_name,
            parent_class=parent_class,
            info="error updating account_name",
        )

    return res

#### sample implementation of update_account_name


In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

try:
    res = await update_account_name(
        auth=token_auth,
        account_id=71,
        account_name=f"domolibrary test account - updated {dt.date.today()}",
        debug_api=False,
    )
    print(res)

except Exception as e:
    print(e)

ResponseGetData(status=200, response={'id': 71, 'userId': 1893952720, 'name': 'domo_creds', 'displayName': 'domolibrary test account - updated 2023-11-10', 'type': 'data', 'valid': True, 'dataProviderType': 'abstract-credential-store', 'credentialsType': 'fields', 'createdAt': 1684447092000, 'createdBy': 1893952720, 'modifiedAt': 1699637687000, 'modifiedBy': 1893952720, 'configurations': {}, 'accountTemplateAuthorizationId': None, 'accountId': 71, 'accountTemplateId': None}, is_success=True, parent_class=None)


In [ ]:
# | export
@gd.route_function
async def create_account(
    auth: dmda.DomoAuth,
    config_body: dict,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
) -> rgd.ResponseGetData:
    url = f"https://{auth.domo_instance}.domo.com/api/data/v1/accounts"

    if debug_api:
        print(url)

    attempt = 1
    res = None

    while attempt <= 3:
        res = await gd.get_data(
            auth=auth,
            url=url,
            method="POST",
            body=config_body,
            debug_api=debug_api,
            session=session,
        )

        if res.is_success:
            return res

        attempt += 1
        await asyncio.sleep(3)

    if not res.is_success:
        raise CreateAccount_Error(
            entity_id=account_name,
            domo_instance=auth.domo_instance,
            status=res.status,
            message=res.response,
            parent_class=parent_class,
            function_name=res.traceback_details.function_name,
        )

    return res

In [ ]:
# | export


@gd.route_function
async def delete_account(
    auth: dmda.DomoAuth,
    account_id: str,
    debug_api: bool = False,
    debug_num_stacks_to_drop=1,
    parent_class: str = None,
    session: httpx.AsyncClient = None,
) -> rgd.ResponseGetData:
    url = f"https://{auth.domo_instance}.domo.com/api/data/v1/accounts/{account_id}"

    if debug_api:
        print(url)

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="DELETE",
        debug_api=debug_api,
        session=session,
        parent_class=parent_class,
        num_stacks_to_drop=debug_num_stacks_to_drop,
    )

    if not res.is_success:
        raise DeleteAccount_Error(
            entity_id=self.id,
            domo_instance=auth.domo_instance,
            status=res.status,
            message=res.response,
            parent_class=parent_class,
            function_name=res.traceback_details.function_name,
        )

    return res

# Account Sharing


In [ ]:
# | export


class ShareAccount_Error(de.DomoError):
    def __init__(
        self, account_id, status, response, domo_instance, function_name, parent_class
    ):
        super().__init__(
            status=status,
            entity_id=account_id,
            message=response,
            domo_instance=domo_instance,
            function_name=function_name,
            parent_class=parent_class,
        )


class ShareAccount:
    pass


class ShareAccount_V1_AccessLevel(ShareAccount, Enum):
    CAN_VIEW = "READ"


class ShareAccount_V2_AccessLevel(ShareAccount, Enum):
    CAN_VIEW = "CAN_VIEW"
    CAN_EDIT = "CAN_EDIT"
    CAN_SHARE = "CAN_SHARE"


def generate_share_account_payload_v1(
    access_level: ShareAccount, user_id: int = None, group_id: int = None
):
    if user_id:
        return {"type": "USER", "id": int(user_id), "permissions": [access_level.value]}
    if group_id:
        return {
            "type": "GROUP",
            "id": int(group_id),
            "permissions": [access_level.value],
        }


def generate_share_account_payload_v2(
    access_level: ShareAccount, user_id: int = None, group_id: int = None
):
    if user_id:
        return {"type": "USER", "id": int(user_id), "accessLevel": access_level.value}

    if group_id:
        return {"type": "GROUP", "id": int(group_id), "accessLevel": access_level.value}

In [ ]:
print(
    generate_share_account_payload_v1(
        user_id=1, access_level=ShareAccount_V1_AccessLevel.CAN_VIEW
    )
)
print(
    generate_share_account_payload_v2(
        user_id=1, access_level=ShareAccount_V2_AccessLevel.CAN_VIEW
    )
)

{'type': 'USER', 'id': 1, 'permissions': ['READ']}
{'type': 'USER', 'id': 1, 'accessLevel': 'CAN_VIEW'}


In [ ]:
# | export
@gd.route_function
async def share_account_v2(
    auth: dmda.DomoAuth,
    account_id: str,
    share_payload: dict,
    debug_api: bool = False,
    parent_class: str = None,
    debug_num_stacks_to_drop=1,
    session: httpx.AsyncClient = None,
):
    url = (
        f"https://{auth.domo_instance}.domo.com/api/data/v2/accounts/share/{account_id}"
    )

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="PUT",
        body=share_payload,
        parent_class=parent_class,
        num_stacks_to_drop=debug_num_stacks_to_drop,
        debug_api=debug_api,
        session=session,
    )

    if res.status == 500 and res.response == "Internal Server Error":
        raise ShareAccount_Error(
            account_id=account_id,
            status=res.status,
            response=f'ℹ️ - {res.response + "| User may already have access to account."}',
            domo_instance=self.domo_instance,
            function_name=res.traceback_details.function_name,
            parent_class=parent_class,
        )

    if not res.status == 200:
        raise ShareAccount_Error(
            account_id=account_id,
            status=res.status,
            response=res.response,
            domo_instance=auth.domo_instance,
            function_name=res.traceback_details.function_name,
            parent_class=parent_class,
        )

    return res

In [ ]:
# | export
@gd.route_function
async def get_account_accesslist_for_v2(
    auth: dmda.DomoAuth,
    account_id: str,
    debug_api: bool = False,
    debug_num_stacks_to_drop: int = 1,
    parent_class: str = None,
    session: httpx.AsyncClient = None,
):
    url = (
        f"https://{auth.domo_instance}.domo.com/api/data/v2/accounts/share/{account_id}"
    )

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="GET",
        debug_api=debug_api,
        num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
        session=session,
    )

    if not res.status == 200:
        GetAccount_NoMatch(
            domo_instance=auth.domo_instance,
            status=res.status,
            function_name=res.traceback_details.function_name,
            parent_class=parent_class,
        )

    return res

#### sample implementation of get_share_account_v2

In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

try:
    res = await get_account_accesslist_for_v2(auth=token_auth, account_id=5)

    for l in res.response["list"]:
        print(l)
except Exception as e:
    print(e)

{'type': 'GROUP', 'id': '1814479647', 'accessLevel': 'CAN_VIEW', 'name': 'Admin Test'}
{'type': 'USER', 'id': '612085674', 'accessLevel': 'CAN_VIEW', 'name': 'Oleksii Zakrevskyi'}
{'type': 'USER', 'id': '587894148', 'accessLevel': 'CAN_VIEW', 'name': 'Bryan Van Kampen'}
{'type': 'USER', 'id': '1893952720', 'accessLevel': 'OWNER', 'name': 'Jae Wilson1'}
{'type': 'USER', 'id': '68216396', 'accessLevel': 'CAN_VIEW', 'name': 'Elliott Leonard'}
{'type': 'USER', 'id': '1256759792', 'accessLevel': 'CAN_VIEW'}


In [ ]:
# | export
# v1 may have been deprecated.  used to be tied to group beta
@gd.route_function
async def share_account_v1(
    auth: dmda.DomoAuth,
    account_id: str,
    share_payload: dict,
    debug_api: bool = False,
    debug_num_stacks_to_drop: int = 1,
    parent_class: str = None,
    session: httpx.AsyncClient = None,
):
    url = (
        f"https://{auth.domo_instance}.domo.com/api/data/v1/accounts/{account_id}/share"
    )

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="PUT",
        body=share_payload,
        num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
        debug_api=debug_api,
        session=session,
    )

    if res.status == 500 and res.response == "Internal Server Error":
        raise ShareAccount_Error(
            account_id=account_id,
            status=res.status,
            response=f'ℹ️ - {res.response + "| User may already have access to account OR may need to execute v2 share API."}',
            domo_instance=auth.domo_instance,
            function_name=res.traceback_details.function_name,
            parent_class=parent_class,
        )

    if not res.status == 200:
        raise ShareAccount_Error(
            account_id=account_id,
            status=res.status,
            response=res.response,
            domo_instance=auth.domo_instance,
            function_name=res.traceback_details.function_name,
            parent_class=parent_class,
        )

    return res

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()
!nbqa black account.ipynb

All done! ✨ 🍰 ✨
1 file left unchanged.
